In [1]:
# https://arxiv.org/pdf/quant-ph/0104030.pdf
# ^^^ Need to be able to prepare arbitrary state!
import numpy as np
import cirq
from functools import reduce
from sympy import *

In [ ]:
# https://arxiv.org/pdf/quant-ph/9503016.pdf

bottom of pg 663[https://rdo.psu.ac.th/sjstweb/journal/27-3/18mathices.pdf]

## Roots of diagonalizable matrices

In this section, we consider an nth root of a diagonalizable matrix.

- Theorem 2.1: Let A be an $m\times m$ complex matrix. If A is diagonalizable, then A has an nth root, for anypositive integer n.

Proof:

Let $A$ be a diagonalizable matrix, i.e., there exists a non-singular matrix S such that $A = SDS^{-1}$where $D=[d_{ij}]_{m\times m}$ is a diagonal matrix.

Let $D^{\frac{1}{n}}=[d_{ij}^{\frac{1}{n}}]_{m \times m}$, where $d_{ij}^{\frac{1}{n}}$ is an n-th root of $d_{ij}$.

So $A = S (D^{\frac{1}{n}})^{n} S^{-1} = (SD^{\frac{1}{n}}S)^{n}$ Therefore an n-th root of A exists.

https://math.stackexchange.com/questions/1168438/the-nth-root-of-the-2x2-square-matrix

In [ ]:
X=Matrix(cirq.X._unitary_())
X

In [ ]:

X = Matrix([[0,1],[1j,0]])

from sympy.physics.quantum import Dagger
Dagger(X)

In [ ]:
np.array(X)

In [ ]:
from numpy import linalg as LA
from sympy import *
from sympy.physics.quantum import Dagger

# SYMPY calculation gets exact diagonal!!! (note matrices are Hermitian)

class Build_V_Gate():
    
    # V^{n} = U
    
    def __init__(self,U, n_power):
        self.U=U
        self.n = n_power
        
        self.D = None
        self.V = None
        self.V_dag = None
        
    def _diagonalise_U(self):

        
        # find diagonal matrix:
        U_matrix = Matrix(self.U)
        self.S, self.D = U_matrix.diagonalize()
        self.S_inv = self.S**-1
        # where U = S D S^{-1}

        if not np.allclose(np.array(self.S*(self.D*self.S_inv), complex), self.U):
            raise ValueError('U != SDS-1')
    
    def Get_V_gate_matrices(self):
        
        if self.D is None:
            self._diagonalise_U()
        
#         D_nth_root = np.power(self.D, 1/self.n)
        D_nth_root = self.D**(1/self.n)
        
#         self.V = np.array(self.S,complex).dot(np.array(D_nth_root,complex)).dot(np.array(self.S_inv,complex))
#         self.V_dag = self.V.conj().transpose()
        self.V = self.S * D_nth_root * self.S_inv
        self.V_dag = Dagger(self.V)
        
        if not np.allclose(reduce(np.matmul, [np.array(self.V, complex) for _ in range(self.n)]), self.U, atol=1e-10):
            raise ValueError('U != V^{}'.format(self.n))       
               
        return np.array(self.V, complex), np.array(self.V_dag, complex)

n_root=2
mat = cirq.X._unitary_()
aa = Build_V_Gate(mat, n_root)
V, V_dag = aa.Get_V_gate_matrices()
reduce(np.matmul, [V for _ in range(n_root)])

In [ ]:
# from numpy import linalg as LA
# # NUMPY VERSION... NOT as good!

# class Build_V_Gate():
    
#     # V^{n} = U
    
#     def __init__(self,U, n_power):
#         self.U=U
#         self.n = n_power
        
#         self.D = None
#         self.V = None
#         self.V_dag = None
        
#     def _diagonalise_U(self):

#         val,vec = np.linalg.eig(self.U)
        
#         #sorting
#         idx = val.argsort()[::-1]   
#         val_sorted = val[idx]
#         vec_sorted = vec[:,idx]
        
#         # find diagonal matrix:
#         vec_sorted_inv = np.linalg.inv(vec_sorted) 
#         self.D = vec_sorted_inv.dot(self.U.dot(vec_sorted))
        
#         self.S=vec_sorted
#         self.S_inv = vec_sorted_inv
#         # where U = S D S^{-1}
        
#         if not np.allclose(self.S.dot(self.D).dot(self.S_inv), self.U):
#             raise ValueError('U != SDS-1')
    
#     def Get_V_gate_matrices(self):
        
#         if self.D is None:
#             self._diagonalise_U()
        
#         D_nth_root = np.power(self.D, 1/self.n)
# #         D_nth_root = np.sqrt(self.D)
        
#         self.V = self.S.dot(D_nth_root).dot(self.S_inv)
#         self.V_dag = self.V.conj().transpose()
        
#         if not np.allclose(reduce(np.matmul, [self.V for _ in range(self.n)]), self.U, atol=1e-1):
#             raise ValueError('U != V^{}'.format(self.n))       
        
#         return self.V, self.V_dag 
        
# mat = cirq.X._unitary_()
# aa = Build_V_Gate(mat, 2)
# V, V_dag = aa.Get_V_gate_matrices()

# np.around(V.dot(V), 3)

In [ ]:
# aa = Build_V_Gate(mat, 4)
# V, V_dag = aa.Get_V_gate_matrices()
# np.around(((V.dot(V)).dot(V)).dot(V), 3)

In [2]:
class My_V_gate(cirq.SingleQubitGate):
    """
    Description

    Args:
        theta (float): angle to rotate by in radians.
        number_control_qubits (int): number of control qubits
    """

    def __init__(self, V, V_dag, dagger_gate = False):
        self.V = V
        self.V_dag = V_dag
        self.dagger_gate = dagger_gate
    def _unitary_(self):
        if self.dagger_gate:
            return self.V_dag
        else:
            return self.V
            
    def num_qubits(self):
        return 1

    def _circuit_diagram_info_(self,args):
        if self.dagger_gate:
            return 'V^{†}'
        else:
            return 'V'

    def __str__(self):
        if self.dagger_gate:
            return 'V^{†}'
        else:
            return 'V'

    def __repr__(self):
        return self.__str__()

In [ ]:
n_root=4
mat = cirq.X._unitary_()
aa = Build_V_Gate(mat, n_root)
V, V_dag = aa.Get_V_gate_matrices()


GATE = My_V_gate(V, V_dag, dagger_gate=True)

circuit = GATE.on(cirq.LineQubit(2))
cirq.Circuit(circuit)

In [ ]:
def int_to_Gray(num, n_qubits):
    # https://en.wikipedia.org/wiki/Gray_code
    
    # print(np.binary_repr(num, n_qubits)) # standard binary form!
    
    # The operator >> is shift right. The operator ^ is exclusive or
    gray_int = num^(num>>1)
    
    return np.binary_repr(gray_int,n_qubits)


### example... note that grey code reversed as indexing from left to right: [0,1,-->, N-1]
for i in range(2**3):
    print(int_to_Gray(i, 3)[::-1])
    
int_to_Gray(6, 4)

In [ ]:
def check_binary_str_parity(binary_str):
    """
    Returns 0 for EVEN parity
    Returns 1 for ODD parity    
    """
    parity = sum(map(int,binary_str))%2
    
    return parity

check_binary_str_parity('0101')

In [ ]:
# NOTE pg 17 of Elementary gates for quantum information
class n_control_U(cirq.Gate):
    """
    """

    def __init__(self, V, V_dag, list_of_control_qubits, list_control_vals, U_qubit):
        self.V = V
        self.V_dag = V_dag
        
        if len(list_of_control_qubits)!=len(list_control_vals):
            raise ValueError('incorrect qubit control bits or incorrect number of control qubits')
        
        self.list_of_control_qubits = list_of_control_qubits
        self.list_control_vals = list_control_vals
        self.U_qubit = U_qubit
        
        self.n_ancilla=len(list_of_control_qubits)
        
    def flip_control_to_zero(self):
        for index, control_qubit in enumerate(self.list_of_control_qubits):
            if self.list_control_vals[index]==0:
                yield cirq.X.on(control_qubit)
            
    def _get_gray_control_lists(self):
        
        grey_cntrl_bit_lists=[]
        n_ancilla = len(self.list_of_control_qubits)
        for grey_index in range(1, 2**n_ancilla):
            
            gray_control_str = int_to_Gray(grey_index, n_ancilla)[::-1] # note reversing order
            control_list = list(map(int,gray_control_str))
            parity = check_binary_str_parity(gray_control_str)
            
            grey_cntrl_bit_lists.append((control_list, parity))
        return grey_cntrl_bit_lists

            
    def _decompose_(self, qubits):
        
        ## flip if controlled on zero
        X_flip = self.flip_control_to_zero()
        yield X_flip
        
        ## perform controlled gate
        n_ancilla = len(self.list_of_control_qubits)

        grey_control_lists = self._get_gray_control_lists()
        
        for control_index, binary_control_tuple in enumerate(grey_control_lists):
            
            binary_control_seq, parity = binary_control_tuple
            control_indices = np.where(np.array(binary_control_seq)==1)[0]
            control_qubit = control_indices[-1]

            if parity==1:
                gate = self.V.controlled(num_controls=1, control_values=[1]).on(self.list_of_control_qubits[control_qubit], self.U_qubit)
#                 gate= 'V'
            else:
                gate = self.V_dag.controlled(num_controls=1, control_values=[1]).on(self.list_of_control_qubits[control_qubit], self.U_qubit)
#                 gate= 'V_dagg'
            
            
            if control_index==0:
                yield gate
#                 print(gate, control_qubit)
            else:
                for c_index in range(len(control_indices[:-1])):
                    yield cirq.CNOT(self.list_of_control_qubits[control_indices[c_index]], self.list_of_control_qubits[control_indices[c_index+1]])
#                     print('CNOT', control_indices[c_index], control_indices[c_index+1])
#                 print(gate, control_qubit)
                yield gate
                for c_index in list(range(len(control_indices[:-1])))[::-1]:
#                     print('CNOT', control_indices[c_index], control_indices[c_index+1])
                    yield cirq.CNOT(self.list_of_control_qubits[control_indices[c_index]], self.list_of_control_qubits[control_indices[c_index+1]])
        
        ## unflip if controlled on zero
        X_flip = self.flip_control_to_zero()
        yield X_flip
        
    def _circuit_diagram_info_(self, args):

#         return cirq.CircuitDiagramInfo(
#             wire_symbols=tuple([*['@' for _ in range(len(self.list_of_control_qubits))],'U']),exponent=1)
        return cirq.protocols.CircuitDiagramInfo(
            wire_symbols=tuple([*['@' if bit==1 else '(0)' for bit in self.list_control_vals],'U']),
            exponent=1)

    def num_qubits(self):
        return len(self.list_of_control_qubits) + 1 #(+1 for U_qubit)
    

In [ ]:
n_control_qubits=6
n_power = 2**(n_control_qubits-2)


In [ ]:
cirq.X._unitary_()

In [ ]:

cirq.LineQubit.range(3)

In [ ]:
### setup V gate ##
n_control_qubits=3
n_power = 2**(n_control_qubits-1)

theta= np.pi/2
# U_GATE_MATRIX = np.array([
#                     [np.cos(theta), np.sin(theta)],
#                     [np.sin(theta), -1* np.cos(theta)]
#                 ])

U_GATE_MATRIX =  cirq.X._unitary_()

get_v_gate_obj = Build_V_Gate(U_GATE_MATRIX, n_power)
V, V_dag = get_v_gate_obj.Get_V_gate_matrices()

V_gate_DAGGER = My_V_gate(V, V_dag, dagger_gate=True)
V_gate = My_V_gate(V, V_dag, dagger_gate=False)

circuit = V_gate_DAGGER.on(cirq.LineQubit(2))
cirq.Circuit(circuit)



## setup n-control-U ###
list_of_control_qubits = cirq.LineQubit.range(3)
list_control_vals=[0,1,0]
U_qubit = cirq.LineQubit(3)

xx = n_control_U(V_gate, V_gate_DAGGER, list_of_control_qubits, list_control_vals, U_qubit)

Q_circuit = cirq.Circuit(cirq.decompose_once(
        (xx(*cirq.LineQubit.range(xx.num_qubits())))))

print(cirq.Circuit((xx(*cirq.LineQubit.range(xx.num_qubits())))))

Q_circuit

In [ ]:
op = cirq.X.controlled(num_controls=3, control_values=[0, 1, 0]).on(*[cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2)], cirq.LineQubit(3))
print(cirq.Circuit(op))
np.isclose(cirq.Circuit(op).unitary(), Q_circuit.unitary(), atol=1e-9)

In [ ]:
### setup V gate ##
n_control_qubits=2
n_power = 2**(n_control_qubits-1)

theta= np.pi/2


U_GATE_MATRIX =  cirq.X._unitary_()

get_v_gate_obj = Build_V_Gate(U_GATE_MATRIX, n_power)
V, V_dag = get_v_gate_obj.Get_V_gate_matrices()

V_gate_DAGGER = My_V_gate(V, V_dag, dagger_gate=True)
V_gate = My_V_gate(V, V_dag, dagger_gate=False)

circuit = V_gate_DAGGER.on(cirq.LineQubit(2))
cirq.Circuit(circuit)



## setup n-control-U ###
list_of_control_qubits = cirq.LineQubit.range(2)
list_control_vals=[0,1]
U_qubit = cirq.LineQubit(2)

xx = n_control_U(V_gate, V_gate_DAGGER, list_of_control_qubits, list_control_vals, U_qubit)

Q_circuit = cirq.Circuit(cirq.decompose_once(
        (xx(*cirq.LineQubit.range(xx.num_qubits())))))

print(cirq.Circuit((xx(*cirq.LineQubit.range(xx.num_qubits())))))

Q_circuit

In [ ]:
op = cirq.X.controlled(num_controls=2, control_values=[0, 1]).on(*[cirq.LineQubit(0), cirq.LineQubit(1)], cirq.LineQubit(2))
print(cirq.Circuit(op))
np.isclose(cirq.Circuit(op).unitary(), Q_circuit.unitary(), atol=1e-6)

In [ ]:
cirq.X.controlled(num_controls=2, control_values=[0,1]).on(
                            *list(cirq.LineQubit.range(3)))

In [ ]:
op = cirq.X.controlled(num_controls=3, control_values=[0, 0, 1]).on(*[cirq.LineQubit(1), cirq.LineQubit(2), cirq.LineQubit(3)], cirq.LineQubit(4))
print(cirq.Circuit(op))


In [7]:
def int_to_Gray(num, n_qubits):
    # https://en.wikipedia.org/wiki/Gray_code
    
    # print(np.binary_repr(num, n_qubits)) # standard binary form!
    
    # The operator >> is shift right. The operator ^ is exclusive or
    gray_int = num^(num>>1)
    
    return np.binary_repr(gray_int,n_qubits)


### example... note that grey code reversed as indexing from left to right: [0,1,-->, N-1]
for i in range(2**3):
    print(int_to_Gray(i, 3)[::-1])
    
int_to_Gray(6, 4)

000
100
110
010
011
111
101
001


'0101'

In [8]:
def check_binary_str_parity(binary_str):
    """
    Returns 0 for EVEN parity
    Returns 1 for ODD parity    
    """
    parity = sum(map(int,binary_str))%2
    
    return parity

check_binary_str_parity('0101')

0

In [9]:
class My_U_Gate(cirq.SingleQubitGate):
    """
    Description

    Args:
        theta (float): angle to rotate by in radians.
        number_control_qubits (int): number of control qubits
    """

    def __init__(self, theta):
        self.theta = theta
    def _unitary_(self):
        Unitary_Matrix = np.array([
                    [np.cos(self.theta), np.sin(self.theta)],
                    [np.sin(self.theta), -1* np.cos(self.theta)]
                ])
        return Unitary_Matrix
    def num_qubits(self):
        return 1

    def _circuit_diagram_info_(self,args):
        # return cirq.CircuitDiagramInfo(
        #     wire_symbols=tuple([*['@' for _ in range(self.num_control_qubits-1)],' U = {} rad '.format(self.theta.__round__(4))]),exponent=1)
        return ' U = {} rad '.format(self.theta.__round__(4))

    def __str__(self):
        return ' U = {} rad '.format(self.theta.__round__(4))

    def __repr__(self):
        return ' U_arb_state_prep'

In [10]:
class My_V_gate(cirq.SingleQubitGate):
    """
    Description

    Args:
        theta (float): angle to rotate by in radians.
        number_control_qubits (int): number of control qubits
    """

    def __init__(self, V_mat, V_dag_mat, dagger_gate = False):
        self.V_mat = V_mat
        self.V_dag_mat = V_dag_mat
        self.dagger_gate = dagger_gate
    def _unitary_(self):
        if self.dagger_gate:
            return self.V_dag_mat
        else:
            return self.V_mat
            
    def num_qubits(self):
        return 1

    def _circuit_diagram_info_(self,args):
        if self.dagger_gate:
            return 'V^{†}'
        else:
            return 'V'

    def __str__(self):
        if self.dagger_gate:
            return 'V^{†}'
        else:
            return 'V'

    def __repr__(self):
        return self.__str__()

In [27]:
from sympy import *
from sympy.physics.quantum import Dagger

# NOTE pg 17 of Elementary gates for quantum information
class n_control_U(cirq.Gate):
    """
    """

    def __init__(self, list_of_control_qubits, list_control_vals, U_qubit, U_cirq_gate, n_control_qubits):
        self.U_qubit = U_qubit
        self.U_cirq_gate = U_cirq_gate
        
        if len(list_of_control_qubits)!=len(list_control_vals):
            raise ValueError('incorrect qubit control bits or incorrect number of control qubits')
        
        self.list_of_control_qubits = list_of_control_qubits
        self.list_control_vals = list_control_vals
        
        self.n_ancilla=len(list_of_control_qubits)
        self.D = None
        self.n_root = 2**(n_control_qubits-1)
        self.n_control_qubits = n_control_qubits
                
        self.V_mat = None
        self.V_dag_mat = None
        
        
    def _diagonalise_U(self):
        
        # find diagonal matrix:
        U_matrix = Matrix(self.U_cirq_gate._unitary_())
        self.S, self.D = U_matrix.diagonalize()
        self.S_inv = self.S**-1
        # where U = S D S^{-1}

        if not np.allclose(np.array(self.S*(self.D*self.S_inv), complex), self.U_cirq_gate._unitary_()):
            raise ValueError('U != SDS-1') 
        
    def Get_V_gate_matrices(self, check=True):
        
        if self.D is None:
            self._diagonalise_U()
        D_nth_root = self.D**(1/self.n_root)
        
        V_mat = self.S * D_nth_root * self.S_inv
        V_dag_mat = Dagger(V_mat)
        
        self.V_mat = np.array(V_mat, complex)
        self.V_dag_mat = np.array(V_dag_mat, complex)
        
        
        if check:
            V_power_n = reduce(np.matmul, [self.V_mat for _ in range(self.n_root)])
            if not np.allclose(V_power_n, self.U_cirq_gate._unitary_()):
                raise ValueError('V^{n} != U') 
        
        
    def flip_control_to_zero(self):
        for index, control_qubit in enumerate(self.list_of_control_qubits):
            if self.list_control_vals[index]==0:
                yield cirq.X.on(control_qubit)
            
    def _get_gray_control_lists(self):
        
        grey_cntrl_bit_lists=[]
        n_ancilla = len(self.list_of_control_qubits)
        for grey_index in range(1, 2**n_ancilla):
            
            gray_control_str = int_to_Gray(grey_index, n_ancilla)[::-1] # note reversing order
            control_list = list(map(int,gray_control_str))
            parity = check_binary_str_parity(gray_control_str)
            
            grey_cntrl_bit_lists.append((control_list, parity))
        return grey_cntrl_bit_lists

            
    def _decompose_(self, qubits):
        if (self.V_mat is None) or (self.V_dag_mat is None):
            self.Get_V_gate_matrices()

        V_gate_DAGGER = My_V_gate(self.V_mat, self.V_dag_mat, dagger_gate=True)
        V_gate = My_V_gate(self.V_mat, self.V_dag_mat, dagger_gate=False)
        
        ## flip if controlled on zero
        X_flip = self.flip_control_to_zero()
        yield X_flip
        
        ## perform controlled gate
        n_ancilla = len(self.list_of_control_qubits)

        grey_control_lists = self._get_gray_control_lists()
        
        for control_index, binary_control_tuple in enumerate(grey_control_lists):
            
            binary_control_seq, parity = binary_control_tuple
            control_indices = np.where(np.array(binary_control_seq)==1)[0]
            control_qubit = control_indices[-1]

            if parity==1:
                gate = V_gate.controlled(num_controls=1, control_values=[1]).on(self.list_of_control_qubits[control_qubit], self.U_qubit)
            else:
                gate = V_gate_DAGGER.controlled(num_controls=1, control_values=[1]).on(self.list_of_control_qubits[control_qubit], self.U_qubit)
            
            if control_index==0:
                yield gate
            else:
                for c_index in range(len(control_indices[:-1])):
                    yield cirq.CNOT(self.list_of_control_qubits[control_indices[c_index]], self.list_of_control_qubits[control_indices[c_index+1]])
                yield gate
                for c_index in list(range(len(control_indices[:-1])))[::-1]:
                    yield cirq.CNOT(self.list_of_control_qubits[control_indices[c_index]], self.list_of_control_qubits[control_indices[c_index+1]])
        
        ## unflip if controlled on zero
        X_flip = self.flip_control_to_zero()
        yield X_flip
        
    def _circuit_diagram_info_(self, args):
#         return cirq.protocols.CircuitDiagramInfo(
#             wire_symbols=tuple([*['@' if bit==1 else '(0)' for bit in self.list_control_vals],'U']),
#             exponent=1)
        return cirq.protocols.CircuitDiagramInfo(
            wire_symbols=tuple([*['@' if bit==1 else '(0)' for bit in self.list_control_vals],self.U_cirq_gate.__str__()]),
            exponent=1)

    def num_qubits(self):
        return len(self.list_of_control_qubits) + 1 #(+1 for U_qubit)
    
    def check_Gate_gate_decomposition(self, tolerance=1e-9):
        """
        function compares single and two qubit gate construction of n-controlled-U 
        against perfect n-controlled-U gate
        
        tolerance is how close unitary matrices are required
        
        """
        
        # decomposed into single and two qubit gates
        decomposed = self._decompose_(None)
        n_controlled_U_quantum_Circuit = cirq.Circuit(decomposed)
        
#         print(n_controlled_U_quantum_Circuit)
        
        # perfect gate
        perfect_circuit_obj = self.U_cirq_gate.controlled(num_controls=self.n_control_qubits, control_values=self.list_control_vals).on(
                            *self.list_of_control_qubits, self.U_qubit)
        
        perfect_circuit = cirq.Circuit(perfect_circuit_obj)
        
#         print(perfect_circuit)
        
        if not np.allclose(n_controlled_U_quantum_Circuit.unitary(), perfect_circuit.unitary(), atol=tolerance):
            raise ValueError('V^{n} != U')
        else:
#             print('Correct decomposition')
            return True
        
    

In [32]:
## setup
n_control_qubits=2
theta= np.pi/4
U_gate = My_U_Gate(theta)

list_of_control_qubits = cirq.LineQubit.range(2)
list_control_vals=[0,1]
U_qubit = cirq.LineQubit(2)


xx = n_control_U(list_of_control_qubits, list_control_vals, U_qubit, U_gate, n_control_qubits)


Q_circuit = cirq.Circuit(cirq.decompose_once(
        (xx(*cirq.LineQubit.range(xx.num_qubits())))))

# NOT decomposing:
print(cirq.Circuit((xx(*cirq.LineQubit.range(xx.num_qubits())))))

# decomposing
Q_circuit

0: ───(0)────────────────
      │
1: ───@──────────────────
      │
2: ─── U = 0.7854 rad ───


0: ───X───@───@───────────@───X───
          │   │           │
1: ───────┼───X───@───────X───@───
          │       │           │
2: ───────V───────V^{†}───────V───

In [29]:
xx.check_Gate_gate_decomposition(tolerance=1e-15)

True

In [20]:
U_single_qubit = My_U_Gate(theta)
perfect_circuit_obj = U_single_qubit.controlled(num_controls=n_control_qubits, control_values=list_control_vals).on(
                            *list_of_control_qubits, U_qubit)
perfect_circuit = cirq.Circuit(perfect_circuit_obj)
perfect_circuit

0: ───(0)────────────────
      │
1: ───@──────────────────
      │
2: ─── U = 0.7854 rad ───

In [15]:
np.allclose(Q_circuit.unitary(), perfect_circuit.unitary(), atol=1e-6)

True